# RAG  based on mistral 7b

## Loading the model

In [1]:
!pip install -q torch datasets
!pip install -q accelerate==0.21.0 \
                peft==0.4.0 \
                bitsandbytes==0.43.1\
                transformers==4.40.1\
                bitsandbytes==0.22.2
                trl==0.4.7

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
import transformers
import huggingface_hub
import bitsandbytes

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
#################################################################
# Tokenizer（pip install --user transformers==4.34.0）
#################################################################
model_name='mistralai/Mistral-7B-Instruct-v0.1'

model_config = transformers.AutoConfig.from_pretrained(
    model_name,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [4]:
#################################################################
# bitsandbytes parameters
#################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [5]:
from transformers import BitsAndBytesConfig
import torch
#################################################################
# Set up quantization config
#################################################################
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

Your GPU supports bfloat16: accelerate training with bf16=True


In [6]:
import torch

print("Torch version:",torch.__version__)

print("Is CUDA enabled?",torch.cuda.is_available())

Torch version: 2.1.0+cu121
Is CUDA enabled? True


In [7]:
from transformers import AutoModelForCausalLM
#################################################################
# Load pre-trained config
#################################################################
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)


`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## RAG based on mistral

#### Create LLM Chain

In [8]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.chains import LLMChain
import transformers
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.1,
    repetition_penalty=1.2,
    return_full_text=True,
    max_new_tokens=700,
)
prompt_template = """
### [INST] 
Instruction: 
Extract and answer using key information from context. Ensure the response is concise, without duplicates, focusing solely on crucial details. 
             
           

### Example
Context: The sun is a star in the center of our solar system.
Question: What is the sun?
Answer: A star at the center of the solar system.

Context: Neil Armstrong was the first person to walk on the moon.
Question: Who was the first person to walk on the moon?
Answer: Neil Armstrong.



### CONTEXT
{context}

### QUESTION:
{question} 

[/INST]
 """

mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# Create prompt from prompt template 
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain 
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

In [9]:
import pandas as pd

# Assume llm_chain is already defined

def process_dataframe(file_path, output_file_path):
    # Load data from Excel file
    df = pd.read_excel(file_path)

    # Iterate through each row of the DataFrame
    for index, row in df.iterrows():
        # Extract context and question from each row, ensuring they are of string type
        context = str(row['Context'])
        question = str(row['Competency Question'])

        # Call the llm_chain method to get the result
        result = llm_chain.invoke({"context": context, "question": question})

        # Save the result in the 'Predicted Answer' column
        df.at[index, 'Predicted Answer'] = result

        # Print the obtained answer
        print(f"Answer for row {index + 1}: {result}")

        # Print the current row being processed to provide progress feedback
        print(f"Processed {index + 1} / {len(df)} rows")

    # Write the modified DataFrame back to a new Excel file
    df.to_excel(output_file_path, index=False)



# Processing the file
process_dataframe('C:/Users/crown/Desktop/RAG mistral 7b v1/train_dataset.xlsx', 
                  'C:/Users/crown/Desktop/RAG mistral 7b v1/v1_training_result.xlsx')  
### These are the input file and out put file




C:\Users\crown\AppData\Roaming\Python\Python39\site-packages\transformers\generation\configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
C:\Users\crown\AppData\Roaming\Python\Python39\site-packages\transformers\generation\configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


行 1 的答案: {'context': 'Materials Design Ontology (MDO) has related Project Open Databases Integration for Materials Design (OPTIMADE); Materials Design Ontology (MDO) has documentation w3id.org/mdo/full/1.0/; Materials Design Ontology (MDO) has description "MDO is an ontology for materials design field, representing the domain knowledge specifically related to solid-state physics and computational materials science."; Open Materials Database has description "The Open Materials Database is a publicly available, searchable database for information on the properties of materials created by people who work in computational materials design research. "; PD Dr. habil.  Thomas Hammerschmidt has expertise in computational materials science; Dr Sarath Menon has expertise in computational materials science; Calphy has discipline computational materials science; Melting temperature computational workflow has discipline computational materials science; Pyscal has discipline computational materials 

C:\Users\crown\AppData\Roaming\Python\Python39\site-packages\transformers\generation\configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


行 2 的答案: {'context': 'Elemental Multiperspective Material Ontology (EMMO) has related Project EMMC-CSA (2016-2019); Elemental Multiperspective Material Ontology (EMMO) has related Project SimDOME (2019-2023); Elemental Multiperspective Material Ontology (EMMO) has related Project MarketPlace (2018-2022); Elemental Multiperspective Material Ontology (EMMO) has related Project VIMMP (2018-2021); Elemental Multiperspective Material Ontology (EMMO) has related Project OntoTrans (2020-2024); Elemental Multiperspective Material Ontology (EMMO) has related Project ReaxPro (2019-2023); Elemental Multiperspective Material Ontology (EMMO) has related Project OntoCommons (2020-2023); Elemental Multiperspective Material Ontology (EMMO) has related Project OYSTER (2017-2021); Elemental Multiperspective Material Ontology (EMMO) has related Project NanoMECommons (2021-2025); Elemental Multiperspective Material Ontology (EMMO) has related Project OpenModel (2021-2025); Elemental Multiperspective Mater

C:\Users\crown\AppData\Roaming\Python\Python39\site-packages\transformers\generation\configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

行 35 的答案: {'context': 'nan', 'question': 'What are the datasets produced in 2022?', 'text': "\nThere isn't enough context provided for me to accurately determine what specific datasets were produced in 2022. Can you please provide more information or clarify your question?"}
已处理 35 / 38 行


C:\Users\crown\AppData\Roaming\Python\Python39\site-packages\transformers\pipelines\base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
C:\Users\crown\AppData\Roaming\Python\Python39\site-packages\transformers\generation\configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


行 36 的答案: {'context': 'Electronic Structure Common Data Format (ESCDF) has description "This project gathers the working documents used to design the specifications\nof the Electronic Structure Common Data Format (ESCDF), an initiative of the\nElectronic Structure Library (ESL) Contributors.\n"; Carolina Materials Database has description "Welcome to Carolina Materials Database created in Jianjun Hu\'s and Ming Hu\'s groups at University of South Carolina. A freely, globally accessible database of 38,716 inorganic material compounds with over 80,000 calculated properties, and growing."', 'question': 'Who is the creators of the "BAM reference data"?', 'text': '\nThe provided text does not mention any creator for the "BAM reference data".'}
已处理 36 / 38 行


C:\Users\crown\AppData\Roaming\Python\Python39\site-packages\transformers\pipelines\base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
C:\Users\crown\AppData\Roaming\Python\Python39\site-packages\transformers\generation\configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


行 37 的答案: {'context': 'Framework for curation and distribution of reference datasets related participant project is Bundesanstalt für Materialforschung und -prüfung (BAM); Ontologies for defects in crystals related participant project is Bundesanstalt für Materialforschung und -prüfung (BAM); NOMAD Metainfo has documentation https://nomad-lab.eu/services/metainfo; NOMAD Metainfo has description "The NOMAD Metainfo stores descriptive and structured information about materials-science data contained in the NOMAD Archive"', 'question': 'What are the datasets published by "BAM"?', 'text': ' Datasets published by BAM include those related to their participant projects on frameworks for curation and distribution of reference datasets, as well as ontologies for defects in crystals.'}
已处理 37 / 38 行
行 38 的答案: {'context': 'nan', 'question': 'Average', 'text': '\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\

In [10]:
df=pd.read_excel('C:/Users/crown/Desktop/RAG mistral 7b v1/v1_training_result.xlsx')

In [11]:
df['Predicted_Answer'] = df['Predicted Answer'].astype(str)

In [12]:
import re

# Add a new column 'answer_text'
df['answer_text'] = ''

# Use regular expression to match the content after 'text':
text_pattern = re.compile(r"'text':(.+?)}", re.DOTALL)

# Iterate through DataFrame and extract content
for index, row in df.iterrows():
    predicted_answer = row['Predicted_Answer']
    match = text_pattern.search(predicted_answer)
    if match:
        # Extract matched content
        answer_text = match.group(1)
    else:
        answer_text = "no answer"  # Set default value if no match is found

    # Store the extracted content in the new column
    df.at[index, 'answer_text'] = answer_text


In [13]:

df['answer_text'] = df['answer_text'].str.replace('\n', ' ', regex=False)

df['answer_text'] = df['answer_text'].str.replace(r'\*\s*', ' ', regex=True)

df['answer_text'] = df['answer_text'].str.replace(r'\\n', ' ', regex=True)

df['answer_text'] = df['answer_text'].str.replace("'", "", regex=False)

df.to_excel('C:/Users/crown/Desktop/RAG mistral 7b v1/v1_training_result.xlsx')

#### Evaluation

In [48]:
import pandas as pd
from rouge import Rouge
from nltk.translate.bleu_score import sentence_bleu
from sentence_transformers import SentenceTransformer, util

# Define paths
excel_file_path = 'C:/Users/crown/Desktop/RAG mistral 7b v1/v1_training_result.xlsx'
output_file_path = 'C:/Users/crown/Desktop/RAG mistral 7b v1/v1_training_result_with_score.xlsx'

# Read Excel file
df = pd.read_excel(excel_file_path, usecols=['Competency Question', 'Context', 'Ground Truth', 'answer_text'])

# Initialize scorers
rouge = Rouge()
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')

# Prepare a dictionary to store scores
scores_dict = {
    "ROUGE-1-f": [], "ROUGE-1-p": [], "ROUGE-1-r": [],
    "ROUGE-2-f": [], "ROUGE-2-p": [], "ROUGE-2-r": [],
    "ROUGE-L-f": [], "ROUGE-L-p": [], "ROUGE-L-r": [],
    "BLEU": [],
    "SBERT-Similarity": []
}

# Calculate scores
for _, row in df.iterrows():
    ground_truth = str(row['Ground Truth']) if not pd.isnull(row['Ground Truth']) else ""
    predicted_answer = str(row['answer_text']) if not pd.isnull(row['answer_text']) else ""
    
    # Calculate ROUGE scores
    if ground_truth and predicted_answer:
        rouge_scores = rouge.get_scores(predicted_answer, ground_truth)[0]
        for key in scores_dict:
            if key.startswith("ROUGE"):
                rouge_type, rouge_metric = key.split('-')[1], key.split('-')[2]
                scores_dict[key].append(rouge_scores['rouge-{}'.format(rouge_type.lower())][rouge_metric])
    else:
        for key in scores_dict:
            if key.startswith("ROUGE"):
                scores_dict[key].append(None)
    
    # Calculate BLEU score
    reference = ground_truth.split()
    candidate = predicted_answer.split()
    bleu_score = sentence_bleu([reference], candidate, weights=(0.25, 0.25, 0.25, 0.25)) if ground_truth and predicted_answer else None
    scores_dict["BLEU"].append(bleu_score)
    
    # Calculate SBERT-Similarity score
    if ground_truth and predicted_answer:
        embeddings1 = sbert_model.encode([ground_truth], convert_to_tensor=True)
        embeddings2 = sbert_model.encode([predicted_answer], convert_to_tensor=True)
        cos_sim = util.pytorch_cos_sim(embeddings1, embeddings2)
        scores_dict["SBERT-Similarity"].append(cos_sim.item())
    else:
        scores_dict["SBERT-Similarity"].append(None)

# Add scores to DataFrame
for key, value in scores_dict.items():
    df[key] = value

# Save to Excel file
df.to_excel(output_file_path, index=False)

print(f"Scores have been calculated and the results are saved to: {output_file_path}")


C:\Users\crown\AppData\Roaming\Python\Python39\site-packages\transformers\generation\configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'context': [Document(page_content='Averaging 45.3 receiving yards per game this season, Alvin Kamara has been a\nfocal point of the aerial attack with Derek Carr at quarterback. Kamara has\naveraged 42.8 receiving yards per game over his career but saw a dip in\nconsecutive years with Andy Dalton at quarterback. Now armed with an upgrade\nunder Center and multiple weapons at wideout to keep the defensive attention\noff him, Kamara is back to elite receiving numbers for a running back. Kamara\nhas recorded 36 or more receiving yards in four of his last five games and has\ntopped 33 or more in five of six games this season. He is being peppered with\ntargets regardless of opponent and should be leaned on again in Week 10. Tap\nthe More on Kamara for Week 10.', metadata={'source': 'https://www.fantasypros.com/2023/11/nfl-week-10-sleeper-picks-player-predictions-2023/', 'title': 'NFL Week 10 Sleeper Picks Player Predictions (2023) | FantasyPros', 'description': 'Welcome to Sleeper Picks. 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load Excel file
output_file_path = 'C:/Users/crown/Desktop/RAG mistral 7b v1/v1_training_result_with_score.xlsx'
df = pd.read_excel(output_file_path)

# Specify columns to compute mean
columns_to_mean = [
    'BLEU', 'SBERT-Similarity',
    'ROUGE-1-f', 'ROUGE-1-p', 'ROUGE-1-r',
    'ROUGE-2-f', 'ROUGE-2-p', 'ROUGE-2-r',
    'ROUGE-L-f', 'ROUGE-L-p', 'ROUGE-L-r'
]

# Compute mean
means = df[columns_to_mean].mean()

# Visualization
ax = means.plot(kind='bar', figsize=(10, 6), color='skyblue', fontsize=13)

plt.title('Average Scores')
plt.xlabel('Metric')
plt.ylabel('Average Score')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for readability
plt.tight_layout()  # Automatically adjust subplot parameters to give specified padding

# Add values above each bar
for i in ax.patches:
    ax.text(i.get_x() + i.get_width() / 2, i.get_height(), f'{i.get_height():.2f}', ha='center', va='bottom')

plt.show()
